In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pymysql
import time

In [2]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'python'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                       db=DB_NAME, charset='utf8')
    
    curs = conn.cursor()

    sql = """insert into joongangjoins(url, title, date, sub_title, contents)
         values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    conn.close()

In [3]:
def joonang(search):
    url = "https://news.joins.com/search/?keyword={search}".format(search=search)
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.get(url)
    return joonangUrlList(driver, search)

In [4]:
def joonangInfo(driver, joonangUrls_List):
    for urlList in joonangUrls_List:
        driver.get(urlList)
        time.sleep(2)
        try:
            newsTitle = ""
            newsTitleList = driver.find_elements_by_css_selector("#article_title")
            newsTitle_temp = newsTitleList[0]
            newsTitle = newsTitle_temp.text
            
            newsDate = ""
            newsDateList = driver.find_elements_by_css_selector(".byline")
            newsDateList_temp = newsDateList[0]
            newsDate = newsDateList_temp.text
            newsDate = newsDate.replace("[중앙일보]", "").replace("입력", "").replace(".", "")[0:10]
            
            newsSubtitle = ""
            newsSubtitleList = driver.find_elements_by_css_selector(".ab_subtitle")
            newsSubtitle_temp = newsSubtitleList[0]
            newsSubtitle = newsSubtitle_temp.text
            
            newsContents = ""
            newsContentsList = driver.find_elements_by_css_selector("#article_body")
            
            for newsContent in newsContentsList:
                newsContents = newsContents + newsContent.text
        except:
            continue
            
        print("Url : ", urlList)
        print("날짜 : ", newsDate)
        print("제목 : " , newsTitle)
        print("부제목 : ", newsSubtitle)
        print("-- 기사 본문 --")
        print(newsContents)
        dbdata = [[urlList, newsTitle, newsDate, newsSubtitle, newsContents]]
        connectDB(dbdata)

In [5]:
def joonangUrlList(driver, search):
    
    for pageNo in range(1, 21):
        url = "https://news.joins.com/Search/JoongangNews?page={pageNo}&Keyword={search}&SortType=New&SearchCategoryType=JoongangNews".format(pageNo = pageNo, search = search)
        driver.get(url)
        time.sleep(2)
        
        joonangUrls_List = []
        joonangUrl = driver.find_elements_by_css_selector(".thumb a")
        for url in joonangUrl:
            joonangUrls_List.append(url.get_attribute("href"))
        joonangInfo(driver, joonangUrls_List)

In [6]:
search = "코로나19"
joonang(search)

Url :  https://news.joins.com/article/23782592
날짜 :    20200521
제목 :  ‘사고력 학원’ CMS, 세계1위 게임엔진기업과 협업하는 이유
부제목 :  이충국 CMS에듀 대표 인터뷰
“언택트 시대 융합·영재교육도 온라인으로”
아바타 활용 실시간 코딩교육 프로그램 11월 출시
-- 기사 본문 --
실시간 코딩교육 프로그램을 개발해 에듀테크 시장에 도전하는 CMS에듀 이충국 대표. 변선구 기자
신종 코로나바이러스 감염증(코로나19) 여파로 ‘언택트’(Untact‧비대면)가 새로운 삶의 방식으로 떠올랐다. 교육은 이런 변화의 중심에 있다. 한국은 물론 미국·유럽 등 대다수의 국가가 온라인 수업으로 등교 수업을 대체했다.  
 
이충국 CMS에듀 대표 인터뷰
“언택트 시대 융합·영재교육도 온라인으로”
아바타 활용 실시간 코딩교육 프로그램 11월 출시
'언택트' 시대에 맞춰 원격수업을 도입하는 교육기업도 늘고 있다. 초·중학생 대상 사고력·영재교육으로 유명한 CMS에듀도 온라인 수학‧코딩교육으로 에듀테크 시장에 도전장을 냈다. 
 
CMS는 최근 게임엔진 분야의 세계 1위 업체 유니티(Unity)와 교육플랫폼 개발 계약을 체결했다. 아바타를 활용한 실시간 코딩교육 프로그램을 출시하기 위해서다. 
 
19일 기자와 만난 이충국 CMS에듀 대표는 “원격수업으로 모든 학생이 스마트 기기와 IT인프라를 갖춘 지금이 온라인 교육으로 확장할 적기라고 판단했다”고 밝혔다.
 
사교육 패러다임도 언택트로 바뀌는 듯하다.
실제로 온라인 수업에 대한 학부모들의 요구가 많다. 수업을 듣고 싶은데 감염 우려에 자녀를 학원에 보내는 데 불안함을 느끼기 때문이다. CMS에듀는 코로나19 사태 2년 전부터 글로벌 시장 공략을 위해 ‘온라인 사고력 클래스’를 준비해왔다. 현장에서 이뤄지는 수학 사고력 수업을 똑같은 방식으로 온라인으로 진행한다. 현재 서울 강남에서 초등학교 2~5학년 학생 100여명을 대상으로 운영하고 있다. 올해 안에 국내는 물론 

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)")